In [1]:
# This code writes the product data table to a TXT file.

product_data = [
    ["Metric", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Annual demand (units)", "10,000", "15,000", "25,000", "2,000", "1,500", "95,000"],
    ["Order cost ($)", "50", "50", "50", "50", "50", "150"],
    ["Price/unit load ($)", "500", "650", "350", "250", "225", "150"],
    ["Space required (m²)", "10", "15", "25", "10", "12", "13"],
    ["Reserve dwell percentage (%)", "0", "0", "20%", "0", "0", "100%"],
    ["Yearly carrying cost rate (%)", "10%", "10%", "10%", "10%", "10%", "10%"]
]

with open("product_data.txt", "w") as f:
    for row in product_data:
        f.write("\t".join(row) + "\n")

In [7]:
# This code writes the flow/product data table to a TXT file.

flow_product_data = [
    ["Flow/Product", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Flow 1 (CD)", "0.0707", "0.0203", "0.0267", "0.3354", "0.4083", "0.0726"],
    ["Flow 2 (R)", "0.0849", "0.2023", "0.0420", "0.5590", "0.6804", "0.0871"],
    ["Flow 3 (RF)", "0.1061", "0.2023", "0.0054", "1.0062", "1.2248", "0.1088"],
    ["Flow 4 (F)", "0.0778", "0.2023", "0.0481", "0.0671", "0.8165", "0.0798"]
]

with open("product_cost_data.txt", "w") as f:
    for row in flow_product_data:
        f.write("\t".join(row) + "\n")

In [8]:
# This code writes the new flow/product quantity table to a TXT file.

flow_product_quantity = [
    ["Flow/Product", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Flow 1 (CD)", "20", "15", "4", "5", "15", "20"],
    ["Flow 2 (R)", "5", "5", "20", "4", "25", "5"],
    ["Flow 3 (RF)", "10", "10", "1", "5", "45", "10"],
    ["Flow 4 (F)", "15", "10", "9", "1", "30", "15"]
]

with open("product_yearly_cost.txt", "w") as f:
    for row in flow_product_quantity:
        f.write("\t".join(row) + "\n")

In [4]:
# This code writes the functional area bounds table to a TXT file.

functional_area_bounds = [
    ["Functional Area", "Lower bound (m²)", "Upper bound (m²)"],
    ["Cross-docking", "0", "15,000"],
    ["Reserve", "35,000", "75,000"],
    ["Forward", "35,000", "75,000"]
]

with open("functional_area_bounds.txt", "w") as f:
    for row in functional_area_bounds:
        f.write("\t".join(row) + "\n")

In [5]:
# This code writes the levels of space available in each functional area to a TXT file.

functional_area_levels = [
    ["Functional Area", "#Levels"],
    ["Cross-docking", "1"],
    ["Reserve", "1"],
    ["Forward", "1"]
]

with open("functional_area_levels.txt", "w") as f:
    for row in functional_area_levels:
        f.write("\t".join(row) + "\n")

In [26]:
# Calculate holding cost per unit for each product and functional area
# Uses yearly carrying cost rate, price per product, and storage cost from product_yearly_cost.txt

import csv

carry_rate = [0.10, 0.10, 0.10, 0.10, 0.10, 0.10]  # yearly carrying cost rate for each product
price = [500, 650, 350, 250, 225, 150]             # price per unit load for each product
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]
functional_areas = ["Cross-docking", "Reserve", "RF", "Forward"]

# Read storage costs from product_yearly_cost.txt
storage = []
with open("product_yearly_cost.txt", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    next(reader)  # skip header
    for row in reader:
        storage.append([float(x) for x in row[1:]])

holding_cost = []  # [functional_area, product, holding_cost]

for f, area in enumerate(functional_areas):
    for p, prod in enumerate(products):
        h_pf = carry_rate[p] * price[p] + storage[f][p]
        holding_cost.append([area, prod, round(h_pf, 2)])

# Write results to file
with open("holding_cost_per_unit.txt", "w") as f:
    f.write("Functional Area\tProduct\tHolding Cost per Unit\n")
    for row in holding_cost:
        f.write("\t".join(map(str, row)) + "\n")

In [27]:
# Calculate EOQ for each product p assigned to flow f
# EOQ = sqrt((2 * D * S) / H)
# D = annual demand, S = order cost, H = holding cost per unit (from previous calculation)

import math

# Data for each product
annual_demand = [10000, 15000, 25000, 2000, 1500, 95000]  # D
order_cost = [50, 50, 50, 50, 50, 150]                   # S

# Read holding cost per unit from holding_cost_per_unit.txt
# We'll use the same order of functional_areas and products as before
holding_cost = []  # 2D list: [functional_area][product]
with open("holding_cost_per_unit.txt", "r") as f:
    next(f)  # skip header
    for i in range(4):  # 4 functional areas
        row = []
        for j in range(6):  # 6 products
            line = f.readline()
            h = float(line.strip().split('\t')[2])
            row.append(h)
        holding_cost.append(row)

# Calculate EOQ for each product in each functional area
eoq = []  # [functional_area, product, EOQ]
functional_areas = ["Cross-docking", "Reserve", "RF", "Forward"]
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]

for f, area in enumerate(functional_areas):
    for p, prod in enumerate(products):
        D = annual_demand[p]
        S = order_cost[p]
        H = holding_cost[f][p]
        EOQ = math.sqrt((2 * D * S) / H) if H > 0 else 0
        eoq.append([area, prod, round(EOQ, 2)])

# Write EOQ results to file
with open("eoq_per_product_flow.txt", "w") as f:
    f.write("Functional Area\tProduct\tEOQ\n")
    for row in eoq:
        f.write("\t".join(map(str, row)) + "\n")

In [29]:
# Calculate average inventory position for each product and functional area as EOQ/2

import math

# Data for each product
annual_demand = [10000, 15000, 25000, 2000, 1500, 95000]  # D
order_cost = [50, 50, 50, 50, 50, 150]                   # S

# Read holding cost per unit from holding_cost_per_unit.txt
holding_cost = []  # 2D list: [functional_area][product]
with open("holding_cost_per_unit.txt", "r") as f:
    next(f)  # skip header
    for i in range(4):  # 4 functional areas
        row = []
        for j in range(6):  # 6 products
            line = f.readline()
            h = float(line.strip().split('\t')[2])
            row.append(h)
        holding_cost.append(row)

# Calculate EOQ and average inventory position for each product in each functional area
avg_inventory = []  # [functional_area, product, avg_inventory]
functional_areas = ["Cross-docking", "Reserve", "RF", "Forward"]
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]

for f, area in enumerate(functional_areas):
    for p, prod in enumerate(products):
        D = annual_demand[p]
        S = order_cost[p]
        H = holding_cost[f][p]
        EOQ = math.sqrt((2 * D * S) / H) if H > 0 else 0
        avg_inv = EOQ / 2
        avg_inventory.append([area, prod, round(avg_inv, 2)])

# Write average inventory results to file
with open("avg_inventory_position.txt", "w") as f:
    f.write("Functional Area\tProduct\tAverage Inventory Position\n")
    for row in avg_inventory:
        f.write("\t".join(map(str, row)) + "\n")

In [30]:
# For Flow 3 (Reserve→Forward): allocation of average inventory to reserve & forward areas
# Let r = reserve_dwell[p] (fraction). For product p assigned to flow 3:
# Reserve area gets r * avg_inventory, Forward area gets (1 - r) * avg_inventory

reserve_dwell = [0, 0, 0.2, 0, 0, 1.0]  # as fractions for each product (from your table)
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]

# Read average inventory for Flow 3 (RF) from avg_inventory_position.txt
avg_inventory_flow3 = []
with open("avg_inventory_position.txt", "r") as f:
    next(f)  # skip header
    for _ in range(12):  # skip first 12 rows (3 flows x 6 products)
        next(f)
    for _ in range(6):  # next 6 rows are Flow 3 (RF)
        line = f.readline()
        avg_inv = float(line.strip().split('\t')[2])
        avg_inventory_flow3.append(avg_inv)

# Allocate average inventory to Reserve and Forward areas for Flow 3
allocation = []  # [Product, Reserve Alloc, Forward Alloc]
for p, prod in enumerate(products):
    r = reserve_dwell[p]
    avg_inv = avg_inventory_flow3[p]
    reserve_alloc = round(r * avg_inv, 2)
    forward_alloc = round((1 - r) * avg_inv, 2)
    allocation.append([prod, reserve_alloc, forward_alloc])

# Write allocation to file
with open("avg_inventory_allocation_flow3.txt", "w") as f:
    f.write("Product\tReserve Allocation\tForward Allocation\n")
    for row in allocation:
        f.write("\t".join(map(str, row)) + "\n")

In [37]:
# Optimization model for warehouse flows and costs with area bounds constraints

import csv
import math
import itertools

# --- PARAMETERS ---
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]
flows = ["Flow 1 (CD)", "Flow 2 (R)", "Flow 3 (RF)", "Flow 4 (F)"]
functional_areas = ["Cross-docking", "Reserve", "Forward"]

annual_demand = [10000, 15000, 25000, 2000, 1500, 95000]  # D[p]
order_cost = [50, 50, 50, 50, 50, 150]                   # Co[p]
space_per_unit = [10, 15, 25, 10, 12, 13]
reserve_dwell = [0, 0, 0.2, 0, 0, 1.0]
B = [1, 1, 1, 1, 1, 1]

# --- READ DATA FILES ---
def read_matrix(filename):
    matrix = [[0]*len(products) for _ in range(len(flows))]
    with open(filename, "r") as f:
        next(f)  # skip header
        for i in range(len(flows) * len(products)):
            area, prod, val = f.readline().strip().split('\t')
            matrix[i // len(products)][i % len(products)] = float(val)
    return matrix


# Handling cost (4x6)
handling_cost = []
with open("product_cost_data.txt", "r") as f:
    next(f)
    for i in range(4):
        row = f.readline().strip().split('\t')[1:]  # skip flow name
        row = [float(x) for x in row]
        handling_cost.append(row)


EOQ = read_matrix("eoq_per_product_flow.txt")

AvgInv = read_matrix("avg_inventory_position.txt")
h_pf = read_matrix("holding_cost_per_unit.txt")

# ...existing code...

# --- READ AREA BOUNDS ---
area_bounds = {}
with open("functional_area_bounds.txt", "r") as f:
    next(f)
    for row in f:
        area, lower, upper = row.strip().split('\t')
        area_bounds[area] = (float(lower.replace(',', '')), float(upper.replace(',', '')))

# --- OPTIMIZATION: find best flow per product ---
product_flow = []  # best flow index for each product
min_costs = []     # minimal cost per product

for p, prod in enumerate(products):
    best_flow = None
    best_cost = float("inf")

    for f in range(len(flows)):
        D = annual_demand[p]
        Co = order_cost[p]
        H = h_pf[f][p]
        eoq = EOQ[f][p]
        avg_inv = AvgInv[f][p]
        hand_cost = handling_cost[f][p]

        scost = avg_inv * H
        hcost = D * hand_cost
        ocost = (D / eoq) * Co if eoq > 0 else 0
        total = hcost + ocost + scost

        if total < best_cost:
            best_cost = total
            best_flow = f

    product_flow.append(best_flow)
    min_costs.append(best_cost)
    #print(f"{prod}: best flow = {flows[best_flow]}, cost = {best_cost:.2f}")

# --- COSTS AND AREA USAGE ---
total_cost = 0
area_CD = 0
area_R = 0
area_F = 0

def project_to_bounds(val, lower, upper):
    return max(lower, min(val, upper))

for p, prod in enumerate(products):
    f = product_flow[p]
    D = annual_demand[p]
    Co = order_cost[p]
    H = h_pf[f][p]
    eoq = EOQ[f][p]
    avg_inv = AvgInv[f][p]
    hand_cost = handling_cost[f][p]

    scost = avg_inv * H
    hcost = D * hand_cost
    ocost = (D / eoq) * Co if eoq > 0 else 0
    total = hcost + ocost + scost

    # Area usage
    if f == 0:  # Flow 1 (CD)
        area_CD += avg_inv * space_per_unit[p]
    elif f == 1:  # Flow 2 (R)
        area_R += avg_inv * space_per_unit[p]
    elif f == 2:  # Flow 3 (RF)
        r = reserve_dwell[p]
        avg_inv_reserve = r * avg_inv
        avg_inv_forward = (1 - r) * avg_inv
        area_R += avg_inv_reserve * space_per_unit[p]
        area_F += avg_inv_forward * B[p] * space_per_unit[p]
    elif f == 3:  # Flow 4 (F)
        area_F += avg_inv * space_per_unit[p]

    total_cost += total

    print(f"{prod} ({flows[f]}): HC=${hcost:.2f}, OC=${ocost:.2f}, SC=${scost:.2f}, Total=${total:.2f}")

# Project areas to their bounds
area_CD = project_to_bounds(area_CD, *area_bounds["Cross-docking"])
area_R = project_to_bounds(area_R, *area_bounds["Reserve"])
area_F = project_to_bounds(area_F, *area_bounds["Forward"])

# --- RESULTS ---
print("\n--- SUMMARY ---")
print(f"Total Cost: ${total_cost:.2f}")
print(f"Area Cross-dock: {area_CD:.2f} m²")
print(f"Area Reserve: {area_R:.2f} m²")
print(f"Area Forward: {area_F:.2f} m²")

# --- CHECK AREA BOUNDS ---
def check_bounds(area_name, area_value):
    lower, upper = area_bounds[area_name]
    if area_value < lower or area_value > upper:
        print(f"⚠️ ADJUSTED: {area_name} area usage set to bounds [{lower:.2f}, {upper:.2f}] m²")
    else:
        print(f"✅ {area_name} area usage is within bounds.")


Product 1 (Flow 2 (R)): HC=$849.00, OC=$3708.10, SC=$3708.10, Total=$8265.20
Product 2 (Flow 1 (CD)): HC=$304.50, OC=$5477.25, SC=$5477.60, Total=$11259.35
Product 3 (Flow 3 (RF)): HC=$135.00, OC=$4743.47, SC=$4743.36, Total=$9621.83
Product 4 (Flow 4 (F)): HC=$134.20, OC=$1140.12, SC=$1140.10, Total=$2414.42
Product 5 (Flow 1 (CD)): HC=$612.45, OC=$1185.77, SC=$1185.75, Total=$2983.97
Product 6 (Flow 2 (R)): HC=$8274.50, OC=$11937.37, SC=$11937.40, Total=$32149.27

--- SUMMARY ---
Total Cost: $66694.05
Area Cross-dock: 1406.49 m²
Area Reserve: 35000.00 m²
Area Forward: 35000.00 m²
